# Automated Occupation Classification and Mapping using Semantic Similarity and Zero-shot Learning
## IMD1107 - Natural Language Processing
### [Dr. Elias Jacob de Menezes Neto](htttps://docente.ufrn.br/elias.jacob)

## Summary

### Keypoints
- **Automated Occupation Classification**: The task involves automatically classifying occupation names into a standardized set of professions using advanced NLP and ML techniques.

- **Challenges**: Key challenges include semantic variability, ambiguity, lack of labeled data, the need for generalization, and efficiency.

- **IBGE Classification**: The Brazilian Institute of Geography and Statistics (IBGE) provides a standardized classification of professions used as a reference.

- **Approach**: The approach includes information retrieval, text classification, data preprocessing, transfer learning, semantic similarity analysis, zero-shot learning, nearest
neighbor search, and domain-specific adaptation.

- **Embedding Models**: Two approaches for embedding models are discussed—an open-source approach using BGE-M3 and a commercial approach using OpenAI's models.

- **Reranking Models**: Reranking models (cross-encoders) are used to refine search results and improve accuracy by considering query-specific context.

- **Zero-shot Learning**: This technique allows the model to classify occupation names into professions without explicit training on those names, leveraging the model's generalization capabilities.

- **Cost Considerations**: The cost of using different models is analyzed, with a focus on balancing performance and budget constraints.

### Takeaways
- **Effective Occupation Classification**: Combining NLP and ML techniques can effectively address the complex problem of occupation classification, ensuring consistency and accuracy in data.

- **Importance of Semantic Similarity**: Analyzing semantic similarity between occupation names and standardized professions is crucial for narrowing down the search space and improving classification accuracy.

- **Utility of Zero-shot Learning**: Zero-shot learning is a powerful tool for handling diverse and evolving job titles, reducing the need for constant retraining.

- **Role of Reranking Models**: Reranking models significantly enhance the relevance and quality of search results by providing query-specific context, which can sometimes eliminate the need for further processing.

- **Cost-Performance Balance**: Choosing between open-source and commercial models involves trade-offs between cost, performance, and control over data handling, necessitating careful evaluation based on specific project requirements.

## Introduction

In the realm of computer science, particularly within Natural Language Processing (NLP) and Machine Learning (ML), many tasks demand sophisticated techniques to achieve the desired outcomes. This is especially true for the task of Automated Occupation Classification and Mapping using Semantic Similarity and Zero-shot Learning," a fascinating application of AI aimed at solving real-world information organization challenges.

### Our Problem

The Public Defender's Office at Rio Grande do Norte (DPE/RN) maintains a database of occupation names that need to be standardized and categorized for statistical analysis and reporting. The challenge is to automatically classify these occupation names into a standardized set of professions, ensuring consistency and accuracy. Given the lack of constraints on the system's input, users can enter any occupation name, making this a challenging task.

The Brazilian Institute of Geography and Statistics (IBGE) has a standardized classification of professions that serves as the reference for this task. The goal is to map each occupation name to the closest matching profession in the IBGE classification. This requires advanced NLP and ML techniques due to the vast diversity and ambiguity in occupation names, as each name may be related to any of the 2557 professions in the IBGE classification.


### The Challenge

The complexity of this task arises from several factors:

1. **Semantic Variability**: Occupation names can vary significantly in wording and structure, making accurate matching to standardized professions difficult.
2. **Ambiguity and Overlap**: Many occupation names are ambiguous or overlap with multiple professions, requiring a nuanced understanding of the context for correct classification.
3. **Lack of Labeled Data**: The absence of labeled occupation-profession pairs hampers the effective training of a supervised model, necessitating novel techniques like zero-shot learning.
4. **Generalization and Adaptation**: The system must generalize knowledge across a wide range of occupation names and adapt to new, unseen examples, ensuring robustness and scalability.
5. **Efficiency and Accuracy**: The system must handle a large volume of occupation names efficiently while maintaining high accuracy in the classification process.

### Our Approach

To address this complex problem, we propose a multi-faceted approach that combines various NLP and ML techniques:

1. **Information Retrieval and Text Classification**: This involves combining elements of information retrieval and text classification to search for the most relevant matches from the IBGE classification for each occupation name. The goal is to map unlabeled occupation names to a standardized set of professions.
2. **Data Preprocessing and Normalization**: Raw occupation names will undergo cleaning and normalization to ensure consistency and quality in the input data, which is crucial for accurate classification.
3. **Transfer Learning**: Utilizing pre-trained language models and embeddings allows us to transfer knowledge from general language tasks to the occupation classification task, reducing the need for task-specific labeled data.
4. **Semantic Similarity Analysis**: By leveraging word embeddings and similarity metrics, we can quantify the semantic similarity between occupation names and standardized professions. This narrows down the search space and improves classification accuracy.
5. **Zero-shot Learning**: Advanced ML techniques will be used to classify occupation names into professions that the model has not been explicitly trained on, allowing the system to generalize knowledge and classify new, unseen occupation names effectively. This explores a phenomenon known as emergence, typical of complex systems like large language models.
6. **Nearest Neighbor Search**: The fundamental matching process can be conceptualized as a k-nearest neighbors search in the embedding space, finding the closest standardized professions to each input occupation name based on their vector representations.
7. **Domain-Specific Adaptation**: The system will adapt general language understanding to the nuances of occupational terminology, balancing broad language knowledge with domain-specific insights to improve classification accuracy.

### Key Components and Techniques

To break down the complexity and sophistication of this task, let's explore into the fundamental components and techniques involved:

- **Information Retrieval and Text Classification**: Combining these elements helps in searching for relevant matches from a standardized classification for each input occupation name.
- **Data Preprocessing and Normalization**: Ensuring that raw data is cleaned and normalized to maintain input data quality.
- **Transfer Learning**: Using pre-trained language models to reduce the need for extensive task-specific training data.
- **Semantic Similarity Analysis**: Utilizing word embeddings and similarity metrics to assess the semantic closeness of occupation names to standardized professions.
- **Zero-shot Learning**: Employing models that can classify data points into categories they haven't been explicitly trained on.
- **Nearest Neighbor Search**: Implementing a k-nearest neighbors search in the embedding space to find the closest matches.
- **Domain-Specific Adaptation**: Tailoring general language understanding to the specific terminology of occupations.

Let's explore how these components interact and contribute to the overall system design and functionality.

In [1]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

from multiprocessing import cpu_count
from typing import Dict, List, Literal, Optional, Any

import joblib
import numpy as np
import pandas as pd
import torch
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI, OpenAI, OpenAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
from transformers import PreTrainedModel, PreTrainedTokenizer
from langchain_ollama import OllamaEmbeddings
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer


# Load environment variables from a .env file. This is necessary to access the OPENAI_API_KEY environment variable.
load_dotenv()

True

In [2]:
# Load the official professions data from a CSV file into a DataFrame
# This file contains professions with CBO (Brazilian Classification of Occupations) codes
official_data = pd.read_csv('data/profissoes_com_cbo.csv')

# Load the unofficial professions data from a CSV file into a DataFrame
# This file contains professions without CBO codes
unofficial_data = pd.read_csv('data/profissoes_sem_cbo.csv')

In [3]:
official_data

,id,codigo,nome
0,1,10105,Oficial general da aeronáutica
1,2,10110,Oficial general do exército
2,3,10115,Oficial general da marinha
3,4,10215,Oficial da marinha
4,5,10205,Oficial da aeronáutica
...,...,...,...
2552,2553,992205,Encarregado geral de operações de conservação ...
2553,2554,992220,Pedreiro de conservação de vias permanentes (e...
2554,2555,992215,Operador de ceifadeira na conservação de vias ...
2555,2556,992210,Encarregado de equipe de conservação de vias p...


In [4]:
unofficial_data

,id,codigo,nome
0,2558,NaN,ADMINISTRADOR DE SISTEMAS OPERACIONAIS
1,2559,NaN,ANALISTA DE INFORMAÇÕES (PESQUISADOR DE INFORM...
2,2560,NaN,EMPREGADO DOMÉSTICO DIARISTA
3,2561,NaN,APOSENTADO
4,2562,NaN,AGRICULTORA
...,...,...,...
8246,10862,NaN,ESTUDANTE - 8 ANOS
8247,10863,NaN,COORDENADOR DE VENDAS
8248,10864,NaN,FARMACÊUTICA - AUXÍLIO DOENÇA
8249,10865,NaN,AUXILIAR DE SERVENTE


In [5]:
# Rename columns in the official_data DataFrame for better readability and consistency
# 'codigo' is renamed to 'code' and 'nome' is renamed to 'text'
official_data.rename({'codigo': 'code', 'nome': 'text'}, axis=1, inplace=True)

# Rename columns in the unofficial_data DataFrame for better readability and consistency
# 'codigo' is renamed to 'code' and 'nome' is renamed to 'text'
unofficial_data.rename({'codigo': 'code', 'nome': 'text'}, axis=1, inplace=True)

# Display the official_data DataFrame to verify the changes
official_data

,id,code,text
0,1,10105,Oficial general da aeronáutica
1,2,10110,Oficial general do exército
2,3,10115,Oficial general da marinha
3,4,10215,Oficial da marinha
4,5,10205,Oficial da aeronáutica
...,...,...,...
2552,2553,992205,Encarregado geral de operações de conservação ...
2553,2554,992220,Pedreiro de conservação de vias permanentes (e...
2554,2555,992215,Operador de ceifadeira na conservação de vias ...
2555,2556,992210,Encarregado de equipe de conservação de vias p...


In [6]:
unofficial_data

,id,code,text
0,2558,NaN,ADMINISTRADOR DE SISTEMAS OPERACIONAIS
1,2559,NaN,ANALISTA DE INFORMAÇÕES (PESQUISADOR DE INFORM...
2,2560,NaN,EMPREGADO DOMÉSTICO DIARISTA
3,2561,NaN,APOSENTADO
4,2562,NaN,AGRICULTORA
...,...,...,...
8246,10862,NaN,ESTUDANTE - 8 ANOS
8247,10863,NaN,COORDENADOR DE VENDAS
8248,10864,NaN,FARMACÊUTICA - AUXÍLIO DOENÇA
8249,10865,NaN,AUXILIAR DE SERVENTE


In [7]:
# Check for duplicate entries in the 'text' column of the official_data DataFrame
# Drop duplicates based on the 'text' column and get the shape of the resulting DataFrame
# Compare the shape of the DataFrame with duplicates removed to the original DataFrame shape
# This helps to confirm that there are no duplicate entries in the 'text' column

official_data.drop_duplicates(subset='text').shape, official_data.shape  # There are no duplicates

((2557, 3), (2557, 3))

In [8]:
# Convert all text entries in the 'text' column of the unofficial_data DataFrame to lowercase
# This ensures uniformity in text data, making it easier to compare and process
unofficial_data['text'] = unofficial_data['text'].str.lower()

official_data['text'] = official_data['text'].str.lower()

In [9]:
# Split the 'text' column of the unofficial_data DataFrame at the first occurrence of the '-' character
# Create a new column 'text_split_1' containing the part before the '-' character, with leading and trailing spaces removed
unofficial_data['text_split_1'] = unofficial_data['text'].apply(lambda x: x.split('-')[0].strip())

# Do the same for the official_data DataFrame
# If there is no '-' character in the text, assign None to 'text_split_2'
unofficial_data['text_split_2'] = unofficial_data['text'].apply(lambda x: x.split('-')[-1].strip() if len(x.split('-')) > 1 else None)

In [10]:
unofficial_data['text_split_1'].value_counts().head(50)

text_split_1
autonomo                       141
do lar                         114
autônomo                        93
estudante                       84
autonoma                        82
desempregado                    74
agricultora                     72
desempregada                    69
aposentado                      45
autônoma                        42
aposentada                      37
agricultor                      34
criança                         29
asg                             25
motorista                       25
pedreiro                        24
vendedora                       20
professora                      20
comerciante                     19
costureira                      17
dona de casa                    17
vendedor                        16
faz bico                        15
manicure                        14
diarista                        13
vendedor ambulante              12
motorista de aplicativo         12
empregada doméstica             12
vigilan

In [11]:
unofficial_data['text_split_2'].value_counts().head(50) # Most data after the - is not useful

text_split_2
desempregado                   131
autonomo                       127
desempregada                   110
autônomo                        87
auxilio brasil                  70
autonoma                        62
bolsa família                   57
bolsa familia                   45
registrado em carteira          44
bpc                             43
autônoma                        39
aposentado                      38
aposentada                      31
                                23
auxilio doença                  21
auxílio brasil                  20
pensionista                     17
clt                             17
carteira assinada               16
mei                             15
seguro desemprego               14
bicos                           13
do lar                          12
diarista                        11
aux. brasil                     10
cargo comissionado              10
asg                             10
comerciante                     10
receben

In [12]:
unofficial_data['text_split_2'].value_counts().head(50).index.tolist()

['desempregado',
 'autonomo',
 'desempregada',
 'autônomo',
 'auxilio brasil',
 'autonoma',
 'bolsa família',
 'bolsa familia',
 'registrado em carteira',
 'bpc',
 'autônoma',
 'aposentado',
 'aposentada',
 '',
 'auxilio doença',
 'auxílio brasil',
 'pensionista',
 'clt',
 'carteira assinada',
 'mei',
 'seguro desemprego',
 'bicos',
 'do lar',
 'diarista',
 'aux. brasil',
 'cargo comissionado',
 'asg',
 'comerciante',
 'recebendo seguro desemprego',
 'estudante',
 'loas',
 'invalidez',
 'informal',
 'vendas',
 'pedreiro',
 'manicure',
 'afastado',
 'agricultora',
 'servidora pública',
 'uber',
 'vigilante',
 'recebe bolsa familia',
 'pensão por morte',
 'supermercado',
 'inss',
 'afastada',
 'motorista',
 'recebe bolsa família',
 'motoboy',
 'segurança']

In [13]:
unofficial_data['text_split_1'].value_counts().head(50).index.tolist()

['autonomo',
 'do lar',
 'autônomo',
 'estudante',
 'autonoma',
 'desempregado',
 'agricultora',
 'desempregada',
 'aposentado',
 'autônoma',
 'aposentada',
 'agricultor',
 'criança',
 'asg',
 'motorista',
 'pedreiro',
 'vendedora',
 'professora',
 'comerciante',
 'costureira',
 'dona de casa',
 'vendedor',
 'faz bico',
 'manicure',
 'diarista',
 'vendedor ambulante',
 'motorista de aplicativo',
 'empregada doméstica',
 'vigilante',
 'auxiliar de cozinha',
 'feirante',
 'pedagoga',
 'pintor',
 'mei',
 'mecânico',
 'empresário',
 'serviços gerais',
 'bpc',
 'servente de pedreiro',
 'servente',
 'autonôma',
 'servidora pública',
 'auxiliar de serviços gerais',
 'mecanico',
 'cabeleireira',
 'atendente',
 'balconista',
 'assistente social',
 'garçonete',
 'eletricista']

In [21]:
unemployed = ['loas',
 'auxílio brasil',
 'aux. brasil',
 'afastado',
 'seguro desemprego',
 'pensão por morte',
 'bpc',
 'aposentada',
 'recebendo seguro desemprego',
 'dona de casa',
 'desempregado',
 'do lar',
 'estudante',
 'criança',
 'aposentado',
 'auxilio brasil',
 'desempregada',
 'recebe bolsa família',
 'pensionista',
 'bolsa família',
 'faz bico',
 'invalidez',
 'inss',
 'recebe bolsa familia',
 'bolsa familia',
 'afastada',
 'auxilio doença',
 ]


self_employed = [
 'autônoma',
 'autônomo',
 'autonôma',
 'informal',
 'autonomo',
 'autonoma',
 
]


In [22]:
def replace_unemployed(text: str) -> str:
    for word in unemployed:
        if word in text:
            return 'desempregado'
    return text


def replace_self_employed(text: str) -> str:
    for word in self_employed:
        if word in text:
            return 'autônomo'
    return text

In [23]:
unofficial_data['text_split_1'].apply(replace_unemployed).apply(replace_self_employed).value_counts().head(50)

text_split_1
desempregado                   1823
autônomo                        856
agricultora                      72
agricultor                       34
asg                              25
motorista                        25
pedreiro                         24
vendedora                        20
professora                       20
comerciante                      19
costureira                       17
vendedor                         16
manicure                         14
diarista                         13
vendedor ambulante               12
empregada doméstica              12
motorista de aplicativo          12
auxiliar de cozinha              11
vigilante                        11
feirante                         11
mei                              10
pedagoga                         10
pintor                           10
mecânico                         10
auxiliar de serviços gerais       9
mecanico                          9
serviços gerais                   9
servente       

In [24]:
unofficial_data['text_split_1'] = unofficial_data['text_split_1'].apply(replace_unemployed).apply(replace_self_employed)

In [30]:
unofficial_data['text_split_1'] = unofficial_data['text_split_1'].apply(lambda x: str(x))

In [31]:
unofficial_data_dedup = unofficial_data.drop_duplicates(subset='text_split_1')
unofficial_data_dedup

,id,code,text,text_split_1,text_split_2
0,2558,NaN,administrador de sistemas operacionais,administrador de sistemas operacionais,None
1,2559,NaN,analista de informações (pesquisador de inform...,analista de informações (pesquisador de inform...,None
2,2560,NaN,empregado doméstico diarista,empregado doméstico diarista,None
3,2561,NaN,aposentado,desempregado,None
4,2562,NaN,agricultora,agricultora,None
...,...,...,...,...,...
8241,10857,NaN,pessoa com deficiência,pessoa com deficiência,None
8243,10859,NaN,agricultor de subsistência,agricultor de subsistência,None
8245,10861,NaN,técnico de segurança eletrônica,técnico de segurança eletrônica,None
8247,10863,NaN,coordenador de vendas,coordenador de vendas,None


In [35]:
official_data["text"] = official_data["text"].apply(lambda x: str(x))

official_data = official_data.query('text.str.len() >= 5')
unofficial_data_dedup = unofficial_data_dedup.query('text_split_1.str.len() >= 5')

In [37]:
# FIXME TODO FIX CODE

official_occupations = official_data["text"].tolist()

unofficial_data_dedup = unofficial_data_dedup[~unofficial_data_dedup.text_split_1.isin(official_occupations)]

unofficial_data_dedup

# # Remove rows from unofficial_data_dedup where 'text_split_1' matches any value in the 'text' column of official_data
# # This ensures that only unique entries not present in the official_data are retained in unofficial_data_dedup
# unofficial_data_dedup = unofficial_data_dedup.query('text_split_1 not in @official_data["text"]')

# # Display the resulting DataFrame to verify the filtering operation
# # This helps to see the updated dataset and confirm that 333 rows were removed because they were already in the official_data dataset
# # The original unofficial_data_dedup had 6060 rows, and after removing duplicates, it has 5727 rows
# unofficial_data_dedup

,id,code,text,text_split_1,text_split_2
3,2561,NaN,aposentado,desempregado,None
4,2562,NaN,agricultora,agricultora,None
6,2564,NaN,auxiliar de creche,auxiliar de creche,None
7,2565,NaN,tecnico judiciário,tecnico judiciário,None
13,2571,NaN,técnico em enfermágem,técnico em enfermágem,None
...,...,...,...,...,...
8241,10857,NaN,pessoa com deficiência,pessoa com deficiência,None
8243,10859,NaN,agricultor de subsistência,agricultor de subsistência,None
8245,10861,NaN,técnico de segurança eletrônica,técnico de segurança eletrônica,None
8247,10863,NaN,coordenador de vendas,coordenador de vendas,None


In [39]:
# Convert the 'text_split_1' column of the unofficial_data_dedup DataFrame to a list
# This list contains the unique unofficial occupation names
unofficial_occupations = unofficial_data_dedup['text_split_1'].tolist()

# Convert the 'text' column of the official_data DataFrame to a list
# This list contains the official occupation names
official_occupations = official_data['text'].tolist()

# Add 'desempregado' (unemployed) to the list of official occupations
# This ensures that the list includes this common status
official_occupations.append('desempregado')

# Add 'aposentado' (retired) to the list of official occupations
# This ensures that the list includes this common status
official_occupations.append('aposentado')

# Add 'autônomo' (self-employed) to the list of official occupations
# This ensures that the list includes this common status
official_occupations.append('autônomo')


In [40]:
len(official_occupations)

2558

## Approaches to Embedding and Language Models

In this section, we'll explore two distinct approaches for implementing our embedding and language models:

### 1. Open-Source Approach (Free)

This method leverages cutting-edge open-source models:

- **Embedding Model**: [BGE-M3](https://huggingface.co/BAAI/bge-m3)
    - BGE-M3 is a state-of-the-art embedding model known for its efficiency and performance in generating high-quality text representations. It works well across a wide range of text lengths and domains.
    - It's particularly well-suited for tasks requiring semantic understanding and similarity comparisons.

- **Language Model (LLM)**: [Llama 3.1](https://llama.meta.com/)
    - Llama 3.1 is an advanced open-source large language model developed by Meta AI.
    - It offers impressive natural language processing capabilities while being freely accessible for research and development.

**Key Advantages**:
- Cost-effective solution for resource-constrained projects
- Full control over model deployment and customization
- Potential for on-premise use, addressing data privacy concerns

**Considerations**:
- May require more computational resources and technical expertise to implement
- Performance might not match commercial alternatives in some scenarios

### 2. Commercial Approach (Paid)

This approach utilizes OpenAI's powerful, proprietary models:

- **Embedding Model**: OpenAI Embedding Model
    - Known for producing high-quality embeddings across a wide range of applications
    - Regularly updated to incorporate the latest advancements in natural language processing

- **Language Model (LLM)**: OpenAI LLMs
    - State-of-the-art performance in various natural language tasks
    - Continuously improved and expanded with new capabilities

**Key Advantages**:
- Cutting-edge performance with minimal setup required
- Scalable solution backed by robust infrastructure
- Regular updates and improvements from OpenAI

**Considerations**:
- Associated costs based on usage
- Potential limitations on customization and fine-tuning
- Dependency on external API and internet connectivity

### Choosing the Right Approach

The selection between these approaches depends on several factors:

- **Budget**: The open-source approach is more cost-effective but may require more initial investment in infrastructure and expertise.
- **Performance Requirements**: OpenAI's models often provide superior performance, especially for complex tasks.
- **Data Privacy**: The open-source approach offers more control over data handling, which may be crucial for sensitive applications.
- **Scalability**: Both approaches can scale, but OpenAI's solution may be easier to scale quickly.
- **Customization Needs**: Open-source models offer more flexibility for fine-tuning and adaptation to specific domains.

> **Note**: It's essential to evaluate the trade-offs between cost, performance, and control when deciding between these approaches. In some cases, a hybrid solution utilizing aspects of both methods might be optimal.

In [41]:
# Initialize the OpenAIEmbeddings model with specified parameters
# 'text-embedding-3-large' is the model name and 3072 is the dimensionality of the embeddings
openai_embeddings_model = OpenAIEmbeddings(model='text-embedding-3-large', dimensions=3072)

# Generate embeddings for the unofficial occupations
# This converts each occupation name in the unofficial_occupations list into a high-dimensional vector
embeddings_unofficial_occupations_openai = openai_embeddings_model.embed_documents(unofficial_occupations)

# Generate embeddings for the official occupations
# This converts each occupation name in the official_occupations list into a high-dimensional vector
embeddings_official_occupations_openai = openai_embeddings_model.embed_documents(official_occupations)

# Create a dictionary mapping each unofficial occupation name to its corresponding embedding vector
corpus_unofficial_occupations_openai = {t: np.array(embeddings_unofficial_occupations_openai[i]) for i, t in enumerate(unofficial_occupations)}

# Create a dictionary mapping each official occupation name to its corresponding embedding vector
corpus_official_occupations_openai = {t: np.array(embeddings_official_occupations_openai[i]) for i, t in enumerate(official_occupations)}

In [42]:
corpus_official_occupations_openai['pedreiro']

array([-0.03263911,  0.05674721, -0.00799964, ..., -0.01241072,
       -0.01652337, -0.0116901 ])

In [43]:
corpus_unofficial_occupations_openai['motorista']

array([-0.00600504,  0.00175588,  0.00823391, ..., -0.00580526,
        0.00596586, -0.00164815])

In [44]:
# Save the unofficial occupations embeddings dictionary to a file using joblib
# This allows for efficient storage and later retrieval of the embeddings
joblib.dump(corpus_unofficial_occupations_openai, 'outputs/occupations/corpus_unofficial_occupations_openai.joblib')

# Save the official occupations embeddings dictionary to a file using joblib
# This allows for efficient storage and later retrieval of the embeddings
joblib.dump(corpus_official_occupations_openai, 'outputs/occupations/corpus_official_occupations_openai.joblib')

['outputs/occupations/corpus_official_occupations_openai.joblib']

In [45]:
# Load the official occupations embeddings dictionary from a file using joblib
# This retrieves the precomputed embeddings for official occupations
corpus_official_occupations_openai = joblib.load('outputs/occupations/corpus_official_occupations_openai.joblib')

# Load the unofficial occupations embeddings dictionary from a file using joblib
# This retrieves the precomputed embeddings for unofficial occupations
corpus_unofficial_occupations_openai = joblib.load('outputs/occupations/corpus_unofficial_occupations_openai.joblib')

In [46]:


# Initialize the OllamaEmbeddings model with the specified model name 'bge-m3'
# This model will be used to generate embeddings for text data
ollama_embeddings_model = OllamaEmbeddings(
    model='bge-m3', 
)

In [28]:
# Generate embeddings for the unofficial occupations using the Ollama embeddings model
# This converts each occupation name in the unofficial_occupations list into a high-dimensional vector
embeddings_unofficial_occupations_ollama = ollama_embeddings_model.embed_documents(unofficial_occupations)

# Generate embeddings for the official occupations using the Ollama embeddings model
# This converts each occupation name in the official_occupations list into a high-dimensional vector
embeddings_official_occupations_ollama = ollama_embeddings_model.embed_documents(official_occupations)

# Create a dictionary mapping each unofficial occupation name to its corresponding embedding vector
corpus_unofficial_occupations_ollama = {t: np.array(embeddings_unofficial_occupations_ollama[i]) for i, t in enumerate(unofficial_occupations)}

# Create a dictionary mapping each official occupation name to its corresponding embedding vector
corpus_official_occupations_ollama = {t: np.array(embeddings_official_occupations_ollama[i]) for i, t in enumerate(official_occupations)}

In [29]:
corpus_official_occupations_ollama['pedreiro']

array([ 0.01888191,  0.02639601, -0.03263669, ..., -0.02803972,
       -0.0587266 ,  0.02553112])

In [30]:
corpus_unofficial_occupations_ollama['motorista']

array([-0.00585191, -0.00846581,  0.01099076, ..., -0.00453714,
       -0.11012239, -0.0018726 ])

In [31]:
# Save the unofficial occupations embeddings dictionary generated by the Ollama model to a file using joblib
# This allows for efficient storage and later retrieval of the embeddings
joblib.dump(corpus_unofficial_occupations_ollama, 'outputs/occupations/corpus_unofficial_occupations_ollama.joblib')

# Save the official occupations embeddings dictionary generated by the Ollama model to a file using joblib
# This allows for efficient storage and later retrieval of the embeddings
joblib.dump(corpus_official_occupations_ollama, 'outputs/occupations/corpus_official_occupations_ollama.joblib')

['outputs/occupations/corpus_official_occupations_ollama.joblib']

In [22]:
# Load the official occupations embeddings dictionary generated by the Ollama model from a file using joblib
# This retrieves the precomputed embeddings for official occupations
corpus_official_occupations_ollama = joblib.load('outputs/occupations/corpus_official_occupations_ollama.joblib')

# Load the unofficial occupations embeddings dictionary generated by the Ollama model from a file using joblib
# This retrieves the precomputed embeddings for unofficial occupations
corpus_unofficial_occupations_ollama = joblib.load('outputs/occupations/corpus_unofficial_occupations_ollama.joblib')

In this section, we will implement the first step of our process: **matching unofficial occupation names to the most relevant professions in the official IBGE classification**. This involves two main tasks: calculating semantic similarity and performing zero-shot learning.

## Step 1: Semantic Similarity Calculation

To begin, we need to find the closest matching profession for each unofficial occupation name. This is achieved by calculating the semantic similarity between the input occupation name and each profession in the IBGE classification. Here's how we will approach this task:

1. **Embedding the Input Occupation Name**:
    - We will convert the input occupation name into a numerical representation known as an embedding. Embeddings capture the semantic meaning of words or phrases in a high-dimensional space, allowing us to measure how similar they are to other words or phrases.

2. **Comparing with IBGE Classifications**:
    - Once we have the embedding for the input occupation name, we will compare it against the embeddings of all professions in the IBGE classification. This comparison involves calculating the distance or similarity between the vectors representing the input occupation name and each IBGE profession.

3. **Identifying Closest Matches**:
    - By determining which IBGE profession embeddings are closest to the input occupation name embedding, we can narrow down the search space to the most relevant matches. This step is crucial for ensuring that we focus on the most likely candidates for the next phase of our process.

In [23]:

def find_most_similar_names(input_text: str, unofficial_occupations: Dict[str, np.ndarray], official_occupations: Dict[str, np.ndarray], top_n: int = 10) -> List[str]:
    """
    Finds the most similar names to a given target name based on cosine similarity.

    Args:
        input_text (str): The name for which to find similar names.
        unofficial_occupations (Dict[str, np.ndarray]): A dictionary mapping names to their unofficial embedding vectors.
        official_occupations (Dict[str, np.ndarray]): A dictionary mapping names to their official embedding vectors.
        top_n (int, optional): The number of most similar names to return. Defaults to 10.

    Returns:
        List[str]: A list of the most similar names.
    """
    # Retrieve the embedding vector for the target name from the unofficial_occupations dictionary
    target_embedding = unofficial_occupations[input_text]
    
    # Calculate cosine similarity between the target name's embedding and all names in official_occupations
    similarity_scores = {
        name: cosine_similarity([target_embedding], [official_occupations[name]])[0][0]
        for name in official_occupations
    }
    
    # Sort the names based on similarity scores in descending order and select the top 'n' names
    most_similar_names = sorted(similarity_scores, key=similarity_scores.get, reverse=True)[:top_n]
    
    # Return the list of most similar names
    return most_similar_names


def get_random_occupation_name(occupation_embeddings: Dict[str, np.ndarray]) -> str:
    """
    Chooses a random occupation name from the occupation_embeddings dictionary.

    Args:
        occupation_embeddings (Dict[str, np.ndarray]): A dictionary mapping occupation names to their embedding vectors.

    Returns:
        str: A randomly chosen occupation name.
    """
    # Choose a random occupation name from the list of occupation names
    random_occupation_name = np.random.choice(list(occupation_embeddings.keys()))
    
    # Return the randomly chosen occupation name
    return random_occupation_name


In [24]:
random_occupation_name = get_random_occupation_name(corpus_unofficial_occupations_openai)
random_occupation_name

'camareira diarista'

In [25]:
random_occupation_candidates_openai = find_most_similar_names(random_occupation_name, corpus_unofficial_occupations_openai, corpus_official_occupations_openai, top_n=10)
random_occupation_candidates_openai

['camareiro de hotel',
 'empregado doméstico diarista',
 'camareira de televisão',
 'camareira de teatro',
 'empregado doméstico faxineiro',
 'empregado doméstico arrumador',
 'camareiro de embarcações',
 'governanta de hotelaria',
 'mordomo de hotelaria',
 'atendente de lavanderia']

In [26]:
random_occupation_candidates_ollama = find_most_similar_names(random_occupation_name, corpus_unofficial_occupations_ollama, corpus_official_occupations_ollama, top_n=10)
random_occupation_candidates_ollama

['camareira de televisão',
 'camareira de teatro',
 'empregado doméstico diarista',
 'camareiro de hotel',
 'barista',
 'modelista de roupas',
 'chefe de portaria de hotel',
 'guarda-roupeira de cinema',
 'recepcionista de banco',
 'camareiro de embarcações']

### Step 1.2: Reranking models

We can use reranking models to improve the accuracy of our search results. Reranking models, also known as cross-encoders, significantly improve the accuracy of search results by reordering documents in terms of their relevance to a given query. This section will discuss the role and importance of rerankers in search systems and how they compare to other models.

We could, for instance, completely avoid going to the second stage of the search process if the reranker is confident enough that the first result is the best one. This would save us a lot of time, as we would not need to go through the zero-shot learning process at all.


#### What is a Reranking Model?

A reranking model is a type of model that takes a query and a document pair and outputs a similarity score. This score is then used to reorder the documents by their relevance to the query.

- **First Stage**: An embedding model (retriever) quickly retrieves a set of potentially relevant documents from a large dataset.
- **Second Stage**: A reranker refines this set by reranking the documents based on their relevance to the query.

> **Why Two Stages?**
> Retrieving a small set of documents from a large dataset is much faster than reranking the entire dataset. Rerankers are slower but more accurate, making this two-stage approach efficient and effective.

#### Why Use Rerankers?

Despite their slower performance, rerankers are much more accurate than embedding models. Here are some key reasons:

- **Information Compression in Bi-encoders**: Bi-encoders compress all possible meanings of a document into a single vector, leading to information loss. This can reduce the accuracy of the retrieval process.

- **Contextual Understanding in Rerankers**: Rerankers analyze the document's meaning specific to the user query at the time of the query, reducing information loss and improving relevance.

#### Comparison: Bi-encoders vs. Rerankers

| Feature | Bi-encoders | Rerankers |
|---------|-------------|-----------|
| **Computation Time** | Fast (pre-computed) | Slow (real-time) |
| **Information Loss** | High (single vector) | Low (raw information) |
| **Contextual Understanding** | Limited (pre-query) | High (query-specific) |

#### How Rerankers Work

When using bi-encoder models with vector search, the heavy transformer computation occurs during the creation of the initial vectors (our Step 1). This means:

1. **Initial Vector Creation**: Pre-compute document vectors using transformers.
2. **Query Processing**:
- Run a single transformer computation to create the query vector.
- Compare the query vector to document vectors using cosine similarity or another lightweight metric.

In contrast, rerankers do not pre-compute vectors. Instead:

1. **Query Processing**:
- Feed the query and a single document into the transformer.
- Run a full transformer inference step.
- Output a similarity score for that document.

<p align="center">
<img src="images/rerankers.webp" alt="Rerankers Diagram" style="width: 50%; height: 50%"/>
</p>


We'll now use the [BGE-Reranker-v2-M3](https://huggingface.co/BAAI/bge-reranker-v2-m3) model to refine our search results and improve the accuracy of our occupation-profession matching process.

In [27]:
# Define the path to the reranker model
reranker_path = 'BAAI/bge-reranker-v2-m3'

# Load the tokenizer for the reranker model from the specified path
reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_path)

# Load the reranker model for sequence classification from the specified path
reranker = AutoModelForSequenceClassification.from_pretrained(reranker_path)

# Set the reranker model to evaluation mode
# This disables dropout and other training-specific behaviors
reranker.eval()

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(8194, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_f

In [28]:


def rerank_occupation_candidates(reranker_model: PreTrainedModel, reranker_tokenizer: PreTrainedTokenizer, target_occupation: str, candidate_occupations: List[str], top_n: int = 5) -> List[str]:
    """
    Reranks a list of occupation candidates based on a reranker model.

    Args:
        reranker_model (PreTrainedModel): The reranker model to use for reranking.
        reranker_tokenizer (PreTrainedTokenizer): The tokenizer for the reranker model.
        target_occupation (str): The occupation name for which candidates are being reranked.
        candidate_occupations (List[str]): The list of occupation candidates to rerank.
        top_n (int, optional): The number of top candidates to return. Defaults to 5.

    Returns:
        List[str]: The reranked list of occupation candidates.
    """
    
    # Create pairs of target occupation and each candidate for reranking
    query_candidate_pairs = [[f'Qual a profissão mais parecida com "{target_occupation}?', candidate] for candidate in candidate_occupations]
    
    # Use the reranker model to rerank the occupation candidates
    with torch.no_grad():
        # Tokenize the pairs for input to the reranker model
        model_inputs = reranker_tokenizer(query_candidate_pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
        
        # Get the logits (scores) from the reranker model
        logits = reranker_model(**model_inputs, return_dict=True).logits.view(-1, ).float()
        
        # Sort the candidates based on the scores in descending order
        sorted_indices = logits.argsort(descending=True)
        sorted_candidates = [query_candidate_pairs[i][1] for i in sorted_indices]
    
    # Return the top 'n' reranked candidates
    return sorted_candidates[:top_n]

In [29]:
rerank_occupation_candidates(
    reranker_model=reranker,
    reranker_tokenizer=reranker_tokenizer,
    target_occupation=random_occupation_name,
    candidate_occupations=random_occupation_candidates_openai,
    top_n=5
)

['empregado doméstico diarista',
 'camareiro de hotel',
 'camareira de teatro',
 'camareira de televisão',
 'camareiro de embarcações']

Reranking is a important step in the search process that can significantly improve the relevance and quality of search results. Here's how we can exploit reranking effectively:

1. **Early Satisfaction of Results**
    - If the reranker produces highly satisfactory results, we may choose to conclude the search process at this stage.
    - This approach can save considerable time and computational resources.

2. **Filtering Irrelevant Results**
    - The reranker can act as a powerful filter, eliminating less relevant or off-topic results.
    - This filtration process creates a more focused dataset for subsequent steps.

3. **Resource Optimization**
    - By potentially bypassing the zero-shot learning (ZSL) stage, we can allocate resources more efficiently.
    - This is particularly beneficial in time-sensitive scenarios or when dealing with large-scale data processing.

> **Note**: The decision to use reranking alone or in combination with ZSL should be based on careful analysis of your specific use case, data characteristics, and performance requirements.
>
> If we feel that the reranking is not enough, we can go back to our ZSL approach. Let's see.

## Step 2: Zero-Shot Learning

After identifying the closest matches through semantic similarity, the next step is to classify the occupation names into professions using zero-shot learning. But what exactly is zero-shot learning, and how does it apply to our occupation classification task?

### Understanding Zero-Shot Learning

Zero-shot learning is a powerful machine learning technique that enables models to classify data they haven't explicitly seen during training. This capability is particularly valuable in our occupation classification task, where we aim to categorize a wide range of potentially novel occupation names.

**Key Characteristics of Zero-Shot Learning:**
- Classifies unseen data points without specific training examples
- Leverages broader knowledge and generalization capabilities
- Particularly useful for handling new, unexpected inputs

> **Comparison with related Techniques:**
> - **Few-Shot Learning:** Model learns from a small number of examples per class
> - **One-Shot Learning:** Model learns from a single example per class
> - **Traditional Supervised Learning:** Model learns from labeled training data.


### The Role of Emergence in Large Language Models

Zero-shot learning in Large Language Models (LLMs) is closely tied to the concept of emergence, a phenomenon where complex systems exhibit unexpected properties or behaviors that arise from simpler components.

**Emergence in Various Systems:**
- **Ant Colonies:** Individual ants following simple rules create complex, organized structures
- **Neural Networks:** Basic artificial neurons combine to produce sophisticated cognitive abilities
- **LLMs:** Vast networks of parameters give rise to unexpected capabilities like zero-shot learning

In LLMs, emergence manifests as the model's ability to perform tasks it wasn't explicitly trained on, such as our occupation classification task.
> To know more about the concept of emergence, consider [watching this video](https://www.youtube.com/watch?v=16W7c0mb-rE).

### Applying Zero-Shot Learning to Occupation Classification

Let's break down how we utilize zero-shot learning in our specific task:

1. **Model Preparation:**
    - Instead of training on every possible occupation name, the model is pre-trained on a diverse corpus of text data
    - This broad knowledge base allows the model to understand language patterns, context, and relationships between concepts

2. **Classification Process:**
    - When presented with a new occupation name, the model:
    - Analyzes the semantic content of the name
    - Compares it to its learned knowledge of professions and job roles
    - Predicts the most appropriate IBGE classification based on semantic similarity

3. **Leveraging Semantic Similarity:**
    - The model uses the semantic similarity scores calculated in the previous step to inform its classification decisions
    - This allows for nuanced understanding of occupation names that might be synonyms or closely related to known professions

4. **Handling Ambiguity:**
    - For occupation names that could fit multiple categories, the model may provide a ranked list of potential classifications
    - This approach acknowledges the innate complexity of occupation categorization

### Advantages and Considerations

**Benefits of Zero-Shot Learning in This Context:**
- Flexibility to classify previously unseen occupation names
- Ability to adapt to evolving job markets and new professions
- Reduced need for constant model retraining as new occupations emerge

**Potential Challenges:**
- Accuracy may vary for highly specialized or region-specific occupations
- The model's performance is dependent on the quality and breadth of its pre-training data
- Misclassifications can occur if the model lacks contextual understanding for certain occupations

Let's use ZSL to create a flexible and powerful occupation classification system capable of handling the diverse and evolving landscape of job titles.

In [30]:
prompt_zsl = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um especialista em classificação de profissões com conhecimento profundo do Código Brasileiro de Ocupações (CBO). "
            "Sua tarefa é classificar uma profissão de entrada em uma das profissões candidatas fornecidas ou determinar que não há correspondência adequada. "
            "Siga estas diretrizes:"
            "\n\n1. Analise cuidadosamente a profissão de entrada e compare-a com a lista de profissões candidatas."
            "\n2. Se houver uma correspondência viável, classifique a entrada na profissão candidata mais apropriada."
            "\n3. Se não houver nenhuma correspondência minimamente viável, retorne NENHUMA."
            "\n4. Considere sinônimos, variações de nomenclatura e descrições similares ao fazer a classificação."
            "\n5. Use seu conhecimento para determinar a correspondência mais precisa, mesmo que não haja uma correspondência exata."
            "\n\nLembre-se: sua precisão na classificação é crucial para o sucesso desta tarefa."
        ),
        ("human", "Profissão de entrada: {text}"),
    ]
)

In [31]:
model_openai = ChatOpenAI(
    model='gpt-4o-mini', # 'model' specifies the model to use, in this case 'gpt-4o-mini'
    temperature=0, # 'temperature' controls the randomness of the model's output, with 0 being more deterministic
    timeout=30, # 'timeout' specifies the maximum time (in seconds) to wait for a response from the model
)

model_llama = ChatOllama(
    model='llama3.1', # 'model' specifies the model to use, in this case 'llama3.1'
    temperature=0, # 'temperature' controls the randomness of the model's output, with 0 being more deterministic
    base_url='http://localhost:11434', # 'base_url' specifies the base URL for the model's API endpoint
    timeout=30, # 'timeout' specifies the maximum time (in seconds) to wait for a response from the model
)

In [32]:
random_occupation_name = get_random_occupation_name(corpus_unofficial_occupations_openai)
random_occupation_candidates_ollama = find_most_similar_names(random_occupation_name, corpus_unofficial_occupations_ollama, corpus_official_occupations_ollama, top_n=10)
random_occupation_candidates_openai = find_most_similar_names(random_occupation_name, corpus_unofficial_occupations_openai, corpus_official_occupations_openai, top_n=10)

# Rerank the occupation candidates using the reranker model

reranked_candidates_ollama = rerank_occupation_candidates(
    reranker_model=reranker,
    reranker_tokenizer=reranker_tokenizer,
    target_occupation=random_occupation_name,
    candidate_occupations=random_occupation_candidates_ollama,
    top_n=5
)

reranked_candidates_openai = rerank_occupation_candidates(
    reranker_model=reranker,
    reranker_tokenizer=reranker_tokenizer,
    target_occupation=random_occupation_name,
    candidate_occupations=random_occupation_candidates_openai,
    top_n=5
)

print(f'Occupation: {random_occupation_name}')
print('Reranked candidates (Ollama):', reranked_candidates_ollama)
print('Reranked candidates (OpenAI):', reranked_candidates_openai)

Occupation: frentesta
Reranked candidates (Ollama): ['frentista', 'barista', 'telefonista', 'lingüista', 'geneticista']
Reranked candidates (OpenAI): ['frentista', 'afretador', 'ortoptista', 'kardexista', 'faxineiro']


In [33]:
# Let's sanity-check what was discarded by the reranker
print('Discarded candidates (Ollama):', [c for c in random_occupation_candidates_ollama if c not in reranked_candidates_ollama])
print('Discarded candidates (OpenAI):', [c for c in random_occupation_candidates_openai if c not in reranked_candidates_openai])

Discarded candidates (Ollama): ['bamburista', 'estatístico', 'perfusionista', 'parteira leiga', 'balanceiro']
Discarded candidates (OpenAI): ['proeiro', 'calafetador', 'fuloneiro', 'engraxate', 'vigia']


In [34]:
# Let's try the model with a prompt

class ProfissaoOpenAI(BaseModel):
    """Converte uma profissão não descrita com CBO para uma profissão descrita com CBO (Classificação Brasileira de Ocupações)"""
    profissao_sem_cbo: str = Field(..., description='Profissão não descrita com CBO')
    profissao_com_cbo: Literal[tuple(reranked_candidates_openai)] = Field(..., description=f'Profissão descrita com CBO. Pode ser uma das seguintes: {reranked_candidates_openai}') 

class ProfissaoOllama(BaseModel):
    """Converte uma profissão não descrita com CBO para uma profissão descrita com CBO (Classificação Brasileira de Ocupações)"""
    profissao_sem_cbo: str = Field(..., description='Profissão não descrita com CBO')
    profissao_com_cbo: Literal[tuple(reranked_candidates_ollama)] = Field(..., description=f'Profissão descrita com CBO. Pode ser uma das seguintes: {reranked_candidates_ollama}')

    # Note that some information from the model is passed to the LLM. I've written those parts in Portuguese to prime the model to answer also in Portuguese.
    # In the class above, everything in Portuguese is passed to the model. I've kept the rest in English, so you can know what will be passed to the model.


zsl_ollama = prompt_zsl | model_llama.with_structured_output(ProfissaoOllama, include_raw=True)
zsl_openai = prompt_zsl | model_openai.with_structured_output(ProfissaoOpenAI, include_raw=True)

In [35]:
result_zsl_ollama = zsl_ollama.invoke(random_occupation_name)
result_zsl_openai = zsl_openai.invoke(random_occupation_name)

In [36]:
result_zsl_ollama

{'raw': AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-25T13:34:23.907414036Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'ProfissaoOllama', 'arguments': {'profissao_com_cbo': 'frentista', 'profissao_sem_cbo': 'frentesta'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 646052603, 'load_duration': 19388017, 'prompt_eval_count': 506, 'prompt_eval_duration': 51593000, 'eval_count': 41, 'eval_duration': 435579000}, id='run-bfaf0ccd-e184-4234-82ba-511f38492d96-0', tool_calls=[{'name': 'ProfissaoOllama', 'args': {'profissao_com_cbo': 'frentista', 'profissao_sem_cbo': 'frentesta'}, 'id': 'de730fa5-7f25-4729-9127-57100866d3e8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 506, 'output_tokens': 41, 'total_tokens': 547}),
 'parsed': ProfissaoOllama(profissao_sem_cbo='frentesta', profissao_com_cbo='frentista'),
 'parsing_error': None}

In [37]:
result_zsl_openai

{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ELdqzaAd4K6XfERAzwR6X4gB', 'function': {'arguments': '{"profissao_sem_cbo":"frentesta","profissao_com_cbo":"frentista"}', 'name': 'ProfissaoOpenAI'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 356, 'total_tokens': 377}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_db4a9208a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a54606e-aee9-4a80-b1b5-6e8f94b3df68-0', tool_calls=[{'name': 'ProfissaoOpenAI', 'args': {'profissao_sem_cbo': 'frentesta', 'profissao_com_cbo': 'frentista'}, 'id': 'call_ELdqzaAd4K6XfERAzwR6X4gB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 356, 'output_tokens': 21, 'total_tokens': 377}),
 'parsed': ProfissaoOpenAI(profissao_sem_cbo='frentesta', profissao_com_cbo='frentista'),
 'parsing_error': None}

In [38]:
result_zsl_ollama['parsed'].profissao_com_cbo

'frentista'

In [39]:
result_zsl_openai['parsed'].profissao_com_cbo

'frentista'

In [40]:
type(model_llama), type(model_openai)

(langchain_ollama.chat_models.ChatOllama,
 langchain_openai.chat_models.base.ChatOpenAI)

In [41]:
# Let's generalize the process to a function

def classify_occupation_zero_shot(occupation_name: str, occupation_candidates: List[str], language_model: Any, prompt_template: Any, include_raw: bool=False) -> Any:
    """
    Classifies an occupation name into a predefined list of occupation candidates using a zero-shot learning approach.

    Args:
        occupation_name (str): The name of the occupation to classify.
        occupation_candidates (List[str]): A list of predefined occupation candidates.
        language_model (Any): The language model to use for classification.
        prompt_template (Any): The prompt template to use with the language model.
        include_raw (bool, optional): Whether to include the raw response from the language model. Defaults to False.

    Returns:
        Any: The response from the language model, including the classified occupation.
    """
    # Add 'NENHUMA' (none) to the list of occupation candidates
    occupation_candidates.append('NENHUMA')

    # Define a Pydantic model for the classification response
    class ProfissaoDataModel(BaseModel):
        """Converte uma profissão não descrita com CBO para uma profissão descrita com CBO (Classificação Brasileira de Ocupações)"""
        ocupacao_sem_cbo: str = Field(..., description='Profissão não descrita com CBO')
        ocupacao_com_cbo: Literal[tuple(occupation_candidates)] = Field(..., description=f'Profissão descrita com CBO que melhor descreve a ocupação sem cbo. Pode ser uma das seguintes: {occupation_candidates}') 
    
    # Note that some information from the model is passed to the LLM. I've written those parts in Portuguese to prime the model to answer also in Portuguese.
    # In the class above, everything in Portuguese is passed to the model. I've kept the rest in English, so you can know what will be passed to the model.

    # Create a runnable pipeline with the prompt template and language model
    classification_pipeline = prompt_template | language_model.with_structured_output(ProfissaoDataModel, include_raw=include_raw).with_retry(
        wait_exponential_jitter=True, # Add jitter to the exponential backoff
        stop_after_attempt=4, # Try at most 4 times
        )
            
    # Invoke the pipeline with the occupation name to get the classification response
    response = classification_pipeline.invoke(occupation_name)
    
    return response

In [42]:
# Let's test the function with a new occupation

random_occupation_name = get_random_occupation_name(corpus_unofficial_occupations_openai)
random_occupation_candidates_ollama = find_most_similar_names(random_occupation_name, corpus_unofficial_occupations_ollama, corpus_official_occupations_ollama, top_n=10)
random_occupation_candidates_openai = find_most_similar_names(random_occupation_name, corpus_unofficial_occupations_openai, corpus_official_occupations_openai, top_n=10)

# Rerank the occupation candidates using the reranker model

reranked_candidates_ollama = rerank_occupation_candidates(
    reranker_model=reranker,
    reranker_tokenizer=reranker_tokenizer,
    target_occupation=random_occupation_name,
    candidate_occupations=random_occupation_candidates_ollama,
    top_n=5
)

reranked_candidates_openai = rerank_occupation_candidates(
    reranker_model=reranker,
    reranker_tokenizer=reranker_tokenizer,
    target_occupation=random_occupation_name,
    candidate_occupations=random_occupation_candidates_openai,
    top_n=5
)

print(f'Occupation: {random_occupation_name}')
print('Reranked candidates (Ollama):', reranked_candidates_ollama)
print('Reranked candidates (OpenAI):', reranked_candidates_openai)



Occupation: tec. em informática
Reranked candidates (Ollama): ['tecnólogo em eletrônica', 'técnico em manutenção de equipamentos de informática', 'tecnólogo em gestão da tecnologia da informação', 'engenheiro de equipamentos em computação', 'tecnólogo em telecomunicações']
Reranked candidates (OpenAI): ['tecnólogo em eletrônica', 'técnico em manutenção de equipamentos de informática', 'técnico eletrônico', 'tecnólogo em gestão da tecnologia da informação', 'engenheiro de equipamentos em computação']


In [43]:
# Classify the occupation using the zero-shot learning approach with the Ollama model
result_zsl_ollama = classify_occupation_zero_shot(
    occupation_name=random_occupation_name,
    occupation_candidates=reranked_candidates_ollama,
    language_model=model_llama,
    prompt_template=prompt_zsl,
    include_raw=False
)

result_zsl_ollama

ProfissaoDataModel(ocupacao_sem_cbo='tec. em informática', ocupacao_com_cbo='técnico em manutenção de equipamentos de informática')

In [44]:
# Classify the occupation using the zero-shot learning approach with the OpenAI model
result_zsl_openai = classify_occupation_zero_shot(
    occupation_name=random_occupation_name,
    occupation_candidates=reranked_candidates_openai,
    language_model=model_openai,
    prompt_template=prompt_zsl,
    include_raw=False
)
result_zsl_openai

ProfissaoDataModel(ocupacao_sem_cbo='tec. em informática', ocupacao_com_cbo='técnico em manutenção de equipamentos de informática')

### Analyzing Zero-Shot Learning Results

Since we're using random examples, let's check the results of our zero-shot classification when I ran it on `atendente de consultorio odontologico`.

#### Model Outputs
- **OpenAI's Model Output:**
```python
ProfissaoDataModel(
ocupacao_sem_cbo='atendente de consultorio odontologico',
ocupacao_com_cbo='recepcionista de consultório médico ou dentário'
)
```
- **Llama 3.1 Output:**
```python
ProfissaoDataModel(
ocupacao_sem_cbo='atendente de consultorio odontologico',
ocupacao_com_cbo='atendente de enfermagem'
)
```

The results show that the two models provided different classifications for the occupation "atendente de consultorio odontologico." This discrepancy highlights the importance of evaluating model outputs and considering the context and domain-specific knowledge when interpreting results.

#### Comparison of Model Capabilities
- **OpenAI Model:**
    - Identified the occupation as **"recepcionista de consultório médico ou dentário"**, which directly relates to a dental office setting.

- **Llama 3.1 Model:**
    - Classified the occupation as **"atendente de enfermagem"**, which typically associates more broadly with healthcare and may not specifically align with a dental office.

#### Interpretation and Context
- The **OpenAI model** seems to provide a more contextually accurate classification, likely due to its extensive training on a larger and more diverse dataset.
- The **Llama 3.1 model** offers a broader interpretation, which might suggest a more generic healthcare role.

#### Model Evaluation and Cost Considerations

Given the assumption that larger models trained on more extensive datasets (like OpenAI's) tend to yield better results, it's important to also consider the associated costs.

- **Embedding Model**:
    - OpenAI Embedding Model (`text-embedding-3-large`) costs 0.13 USD per 1,000,000 tokens. For our task, encoding the names of occupations is negligible in cost.

- **Language Model**:
    - OpenAI LLM `gpt-4o-mini` costs 0.150 USD per 1,000,000 tokens of input and 0.60 USD per 1,000,000 tokens of output. Each request includes:
        - Occupation name
        - List of candidate occupations
        - Schema for the Pydantic model
        - Custom prompt created for classification

This can become expensive, depending on the number of requests required.

#### Estimating Classification Costs
To estimate the cost of classifying our dataset, consider the following points:
- Number of occupations to classify.
- Average token count per request (including input and output).

Let's calculate the average cost for classifying a single occupation name using the OpenAI LLM `gpt-4o-mini` model.

In [45]:
# Classify the occupation using the zero-shot learning approach with the OpenAI model
result_zsl_openai = classify_occupation_zero_shot(
    occupation_name=random_occupation_name,
    occupation_candidates=reranked_candidates_openai,
    language_model=model_openai,
    prompt_template=prompt_zsl,
    include_raw=True # Include the raw response from the model, which contains the tokens used in the call
)

price_of_call_input = (0.150 / 1000000) * result_zsl_openai['raw'].usage_metadata['input_tokens']
price_of_call_output = (0.60 / 1000000) * result_zsl_openai['raw'].usage_metadata['output_tokens']
estimated_price = (price_of_call_input + price_of_call_output) * len(corpus_unofficial_occupations_ollama)

print(f'Estimated price for classifying all {len(corpus_unofficial_occupations_ollama)} occupations: ${estimated_price:.2f}')

Estimated price for classifying all 3969 occupations: $0.32


`Less than 50 cents for the entire dataset is not a bad deal!` Let's proceed with the classification process.

In [47]:
def full_zero_shot_occupation_classification(
    input_occupation: str,
    reranker_model: PreTrainedModel,
    reranker_tokenizer: PreTrainedTokenizer,
    language_model: Any,
    prompt_template: Any,
    unofficial_occupation_embeddings: Dict[str, np.ndarray],
    official_occupation_embeddings: Dict[str, np.ndarray],
    include_raw_output: bool = False,
    initial_candidate_count: int = 10,
    final_candidate_count: int = 5,
) -> Dict[str, Any]:
    """
    Performs a full zero-shot learning pipeline for occupation classification.

    Args:
        input_occupation (str): The occupation name to classify.
        reranker_model (PreTrainedModel): The model used for reranking candidates.
        reranker_tokenizer (PreTrainedTokenizer): The tokenizer for the reranker model.
        language_model (Any): The language model used for zero-shot classification.
        prompt_template (Any): The template for generating prompts.
        unofficial_occupation_embeddings (Dict[str, np.ndarray]): Embeddings of unofficial occupation names.
        official_occupation_embeddings (Dict[str, np.ndarray]): Embeddings of official occupation names.
        include_raw_output (bool, optional): Whether to include raw model output. Defaults to False.
        initial_candidate_count (int, optional): Number of initial candidates to consider. Defaults to 10.
        final_candidate_count (int, optional): Number of final candidates after reranking. Defaults to 5.

    Returns:
        Dict[str, Any]: The classification result.
    """

    # Step 1: Find the most similar occupation names
    similar_occupation_candidates = find_most_similar_names(
        input_occupation, 
        unofficial_occupation_embeddings, 
        official_occupation_embeddings, 
        top_n=initial_candidate_count
    )

    # Step 2: Rerank the occupation candidates using the reranker model
    reranked_occupation_candidates = rerank_occupation_candidates(
        reranker_model=reranker_model,
        reranker_tokenizer=reranker_tokenizer,
        target_occupation=input_occupation,
        candidate_occupations=similar_occupation_candidates,
        top_n=final_candidate_count
    )

    # Step 3: Classify the occupation using zero-shot learning
    zero_shot_classification_result = classify_occupation_zero_shot(
        occupation_name=input_occupation,
        occupation_candidates=reranked_occupation_candidates,
        language_model=language_model,
        prompt_template=prompt_template,
        include_raw=include_raw_output
    )

    return zero_shot_classification_result

In [48]:
full_zero_shot_occupation_classification(
    input_occupation=random_occupation_name,
    reranker_model=reranker,
    reranker_tokenizer=reranker_tokenizer,
    language_model=model_openai,
    prompt_template=prompt_zsl,
    unofficial_occupation_embeddings=corpus_unofficial_occupations_openai,
    official_occupation_embeddings=corpus_official_occupations_openai,
    include_raw_output=False,
    initial_candidate_count=10,
    final_candidate_count=5,
)

ProfissaoDataModel(ocupacao_sem_cbo='tec. em informática', ocupacao_com_cbo='técnico em manutenção de equipamentos de informática')

In [49]:
def wrapper_zsl_openai(input_occupation):
    try:
        return full_zero_shot_occupation_classification(
            input_occupation=input_occupation,
            reranker_model=reranker,
            reranker_tokenizer=reranker_tokenizer,
            language_model=model_openai,
            prompt_template=prompt_zsl,
            unofficial_occupation_embeddings=corpus_unofficial_occupations_openai,
            official_occupation_embeddings=corpus_official_occupations_openai,
            include_raw_output=False,
            initial_candidate_count=15,
            final_candidate_count=5,
        )
    except Exception as e:
        print(f'Error on {input_occupation}: {e}')

def wrapper_zsl_ollama(input_occupation):
    try:
        return full_zero_shot_occupation_classification(
            input_occupation=input_occupation,
            reranker_model=reranker,
            reranker_tokenizer=reranker_tokenizer,
            language_model=model_llama,
            prompt_template=prompt_zsl,
            unofficial_occupation_embeddings=corpus_unofficial_occupations_ollama,
            official_occupation_embeddings=corpus_official_occupations_ollama,
            include_raw_output=False,
            initial_candidate_count=15,
            final_candidate_count=5,
        )
    except Exception as e:
        print(f'Error on {input_occupation}: {e}')

In [50]:
wrapper_zsl_ollama('cabista')

ProfissaoDataModel(ocupacao_sem_cbo='cabista', ocupacao_com_cbo='cableador')

In [51]:
wrapper_zsl_openai('cabista')

ProfissaoDataModel(ocupacao_sem_cbo='cabista', ocupacao_com_cbo='cableador')

In [52]:
unofficial_data_dedup

,id,code,text,text_split_1,text_split_2
3,2561,NaN,aposentado,desempregado,None
4,2562,NaN,agricultora,agricultora,None
6,2564,NaN,auxiliar de creche,auxiliar de creche,None
7,2565,NaN,tecnico judiciário,tecnico judiciário,None
13,2571,NaN,técnico em enfermágem,técnico em enfermágem,None
...,...,...,...,...,...
8241,10857,NaN,pessoa com deficiência,pessoa com deficiência,None
8243,10859,NaN,agricultor de subsistência,agricultor de subsistência,None
8245,10861,NaN,técnico de segurança eletrônica,técnico de segurança eletrônica,None
8247,10863,NaN,coordenador de vendas,coordenador de vendas,None


In [53]:
unofficial_data_dedup['ollama_labels'] = unofficial_data_dedup['text_split_1'].apply(wrapper_zsl_ollama)

Error on agricultora: 1 validation error for ProfissaoDataModel
ocupacao_com_cbo
  unexpected value; permitted: 'tratorista agrícola', 'técnico agrícola', 'trabalhador no cultivo de forrações', 'trabalhador no cultivo de árvores frutíferas', 'trabalhador no cultivo de plantas ornamentais', 'NENHUMA' (type=value_error.const; given=trabalhadora no cultivo de plantas ornamentais; permitted=('tratorista agrícola', 'técnico agrícola', 'trabalhador no cultivo de forrações', 'trabalhador no cultivo de árvores frutíferas', 'trabalhador no cultivo de plantas ornamentais', 'NENHUMA'))
Error on motorista de aplicativo: 1 validation error for ProfissaoDataModel
ocupacao_com_cbo
  unexpected value; permitted: 'ajudante de motorista', 'engenheiro de aplicativos em computação', 'motorista de táxi', 'motorista de carro de passeio', 'condutor de veículos a pedais', 'NENHUMA' (type=value_error.const; given=engenheiro de aplicativos em computácao; permitted=('ajudante de motorista', 'engenheiro de aplica

/tmp/ipykernel_1119035/2228753918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unofficial_data_dedup['ollama_labels'] = unofficial_data_dedup['text_split_1'].apply(wrapper_zsl_ollama)


In [54]:
unofficial_data_dedup['openai_labels'] = unofficial_data_dedup['text_split_1'].apply(wrapper_zsl_openai)

/tmp/ipykernel_1119035/1728829878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unofficial_data_dedup['openai_labels'] = unofficial_data_dedup['text_split_1'].apply(wrapper_zsl_openai)


In [55]:
unofficial_data_dedup['openai_labels']

3       ocupacao_sem_cbo='desempregado' ocupacao_com_c...
4       ocupacao_sem_cbo='agricultora' ocupacao_com_cb...
6       ocupacao_sem_cbo='auxiliar de creche' ocupacao...
7       ocupacao_sem_cbo='tecnico judiciário' ocupacao...
13      ocupacao_sem_cbo='técnico em enfermágem' ocupa...
                              ...                        
8241    ocupacao_sem_cbo='pessoa com deficiência' ocup...
8243    ocupacao_sem_cbo='agricultor de subsistência' ...
8245    ocupacao_sem_cbo='técnico de segurança eletrôn...
8247    ocupacao_sem_cbo='coordenador de vendas' ocupa...
8249    ocupacao_sem_cbo='auxiliar de servente' ocupac...
Name: openai_labels, Length: 3969, dtype: object

In [56]:
def get_occupation_label(occupation):
    try:
        return occupation.ocupacao_com_cbo
    except:
        return None
    
unofficial_data_dedup['openai_labels'] = unofficial_data_dedup['openai_labels'].apply(get_occupation_label)
unofficial_data_dedup['ollama_labels'] = unofficial_data_dedup['ollama_labels'].apply(get_occupation_label)

/tmp/ipykernel_1119035/4264424436.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unofficial_data_dedup['openai_labels'] = unofficial_data_dedup['openai_labels'].apply(get_occupation_label)
/tmp/ipykernel_1119035/4264424436.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unofficial_data_dedup['ollama_labels'] = unofficial_data_dedup['ollama_labels'].apply(get_occupation_label)


In [57]:
unofficial_data_dedup.sample(20)

,id,code,text,text_split_1,text_split_2,ollama_labels,openai_labels
588,3148,NaN,educador físico,educador físico,None,professor de educação física do ensino fundame...,NENHUMA
3012,5620,NaN,professor da ufrn efetivo,professor da ufrn efetivo,None,professor de direito do ensino superior,NENHUMA
3686,6299,NaN,auxiliar de pasteleiro,auxiliar de pasteleiro,None,auxiliar nos serviços de alimentação,NENHUMA
3061,5669,NaN,auxiliar de gestão,auxiliar de gestão,None,NENHUMA,assistente administrativo
6539,9155,NaN,alimentos do pai,alimentos do pai,None,NENHUMA,NENHUMA
5860,8476,NaN,atende de padaria,atende de padaria,None,padeiro,atendente de lanchonete
8210,10826,NaN,operador ambiental,operador ambiental,None,operador de colhedor florestal,técnico de controle de meio ambiente
8061,10677,NaN,auxiliar de cobrador,auxiliar de cobrador,None,cobrador externo,localizador (cobrador)
6592,9208,NaN,auxiliar de armazem,auxiliar de armazem,None,NENHUMA,armazenista
7140,9756,NaN,agente de cultura,agente de cultura,None,NENHUMA,produtor cultural


In [58]:
unofficial_data_dedup.to_parquet('outputs/occupations/unofficial_data_dedup.parquet')

In [59]:
agreements = (unofficial_data_dedup['openai_labels'] == unofficial_data_dedup['ollama_labels']).sum() / len(unofficial_data_dedup)
print(f'Agreement between OpenAI and Ollama: {agreements:.2%}')

Agreement between OpenAI and Ollama: 48.20%
